MNIST를 활용한 딥러닝 구현

In [25]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # 물리적 순서대로 정렬
os.environ["CUDA_VISIBLE_DEVICES"] = "6"        # 나는 6번 GPU만 보이게 하겠다!

import torch
print(f"현재 사용 가능한 GPU 개수: {torch.cuda.device_count()}")
print(f"현재 선택된 GPU 번호: {torch.cuda.current_device()}")
print(f"장치 이름: {torch.cuda.get_device_name(0)}")

현재 사용 가능한 GPU 개수: 1
현재 선택된 GPU 번호: 0
장치 이름: NVIDIA GeForce RTX 3090


In [26]:
#### 매!!우!! 중요한 내용임!!! 이해하기

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5) # 1개의 흑백사진과, 6개의 필터. 그리고 그 필터 크기는 5*5크기
        self.conv2 = nn.Conv2d(6, 16, 5) # 6개의 입력 채널, 16개의 출력을 해주는 필터들. 그리고 그 필터 5*5크기
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension. 필요한 정보 조합해서 400개를 120개로 줄임. 학습된 가중치 기준.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, input):
        # Convolution layer C1: 1 input image channel, 6 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a Tensor with size (N, 6, 28, 28), where N is the size of the batch
        c1 = F.relu(self.conv1(input)) # 입력을 6개의 필터에 통과시키고, 각각 레루 적용
        # Subsampling layer S2: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 6, 14, 14) Tensor
        s2 = F.max_pool2d(c1, (2, 2)) # 2*2 크기의 필터, 근데 2칸씩 이동하고, 그 안에서 가장 큰 수만 남기는 필터. 사이즈는 가로세로 1/2씩
        # Convolution layer C3: 6 input channels, 16 output channels,
        # 5x5 square convolution, it uses RELU activation function, and
        # outputs a (N, 16, 10, 10) Tensor
        c3 = F.relu(self.conv2(s2)) # s2를 16개의 필터 통과시키고, 렐류 적용.
        # Subsampling layer S4: 2x2 grid, purely functional,
        # this layer does not have any parameter, and outputs a (N, 16, 5, 5) Tensor
        s4 = F.max_pool2d(c3, 2) # c3를 2*2 필터로 1/2 크기로 줄임.
        # Flatten operation: purely functional, outputs a (N, 400) Tensor
        s4 = torch.flatten(s4, 1) # -->> 이걸 한 줄로 펴줌. 단, N의 정보는 빼고.
        # Fully connected layer F5: (N, 400) Tensor input,
        # and outputs a (N, 120) Tensor, it uses RELU activation function
        f5 = F.relu(self.fc1(s4)) # 그 정보를 줄이고 렐루 적용. 근거는 데이터로부터 학습된 가중치 기준으로.
        # Fully connected layer F6: (N, 120) Tensor input,
        # and outputs a (N, 84) Tensor, it uses RELU activation function
        f6 = F.relu(self.fc2(f5)) # 다시 줄이고 렐루 적용
        # Fully connected layer OUTPUT: (N, 84) Tensor input, and
        # outputs a (N, 10) Tensor
        output = self.fc3(f6) # 다시 줄이고 렐루 적용
        return output
# 굉장히 큰 수를 숫자 10개로 줄였다. 이제 10개의 이미지 종류중에서 뭐에 해당하는지 맞춰볼 수 있다.

net = Net().to(device)
print(net)

Using device: cuda
Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [27]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight. 필터 6개이고, 각 1개당 5*5이다. 

10
torch.Size([6, 1, 5, 5])


In [28]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([ # 합성함수를 해 준다. 매번 두 함수 써야 했는데, 두 개 함성해줌.
    transforms.Resize(32), # 고무줄처럼 늘려서 다시 작성함. 
    transforms.ToTensor(), # 픽셀값을 0~1사이의 정규화된 float으로 변환해줌. + 채널(흑백은 1. 이 실험에서 바뀔 일 없음.) 차원도 추가함. 
])

train_dataset = torchvision.datasets.MNIST( # MNIST 라는 클래스를 따라갈 수 있도록 새로 지정해 준 것일 뿐이다. 
    root='./data',
    train=True,
    download=True,
    transform=transform # MNIST 클래스의 생성자 인자 중, transform 에는 우리가 위에 만들어둔 transform을 끼워넣자!
)

test_dataset = torchvision.datasets.MNIST( # 이미지 꺼내는 방법을 정의한 객체
    root='./data',
    train=False, # 이건 테스트 셋임.
    download=True,
    transform=transform
)

train_loader = torch.utils.data.DataLoader( # 클래스를 호출해서 만든 객체를 담음. dataset과 다르게 dataloader는 여러 장을 묶어서 반복하는 법.
    train_dataset,
    batch_size=64,
    shuffle=True
) # 매 epoch마다 전체 데이터 섞고, 앞에 64개 잘러서 묶음.

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
) # 재현성 때문에 안 섞고, 어차피 모든 데이터를 사용할 거라 안 섞는 이유가 됨.


In [ ]:
criterion = nn.CrossEntropyLoss() # -->> 이 부분. 소프트 맥스로 확률이 몰리게끔 도와줌.

for images, labels in train_loader:
    images = images.to(device) # 같은 GPU위에 올라가 있어야 연산이 가능한데, CPU에 올라가 있길래 변경해줌.
    labels = labels.to(device)

    outputs = net(images)     # (N, 10)
    loss = criterion(outputs, labels)  # labels: (N,), 정수 0~9
    print(loss)
    break

tensor(2.3058, device='cuda:0', grad_fn=<NllLossBackward0>)


In [34]:
# 학습에서 쓰이는 grd는 그 step동안 backward로 만들어진 grad의 합이다. 그래서 계속 누적이 되는 것임.
net.zero_grad()     # zeroes the gradient buffers of all parameters
# 이걸 이해하면, 이후에 옵티마이저를 더 쉽게 이해할 수 있음.
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward() # 61,706차원의 가중치공간에서, 각각의 축으로 미분한 값을 모아 그라디언트 벡터를 61,706차원에 하나 만들었다.
#그리고 아미지 N장 중 미니배치 4장만 골라서 그라디언트를 구하면, 4개 함수의 그라디언트의 합으로 loss.backward가 만들어지고, 전파하면 됨.
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0008, -0.0018,  0.0016,  0.0029, -0.0007, -0.0008], device='cuda:0')


In [35]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate) # 실제로 이렇게 돌아가지만, 다른 모델에서도 똑같이 돌아가게 하기 위해서 optimizer 사용.

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9) #모멘텀은 관성임.

for images, labels in train_loader: # train_loader에서 images, labels를 하나씩 꺼내서 반복하겠다는 의미. 
    images = images.to(device) # 이미지 60,000장인데, 배치크기 64니까 총 938번 돌음.
    labels = labels.to(device)

    outputs = net(images)
    loss = criterion(outputs, labels)

In [ ]:
num_epochs = 5 # epoch는 전체 데이터셋을 한 번 도는 걸 몇번하는지이다. 
                # 부연 설명을 하자면, 6만장을 앞에서부터 64개씩 끊어서 사용하고 셔플 후 다시 6만장을 끊어 사용함. 이걸 5번 반복함.
for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()          # grad 초기화
        outputs = net(images)          # forward
        loss = criterion(outputs, labels)
        loss.backward()                # backward
        optimizer.step()               # update

        running_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {running_loss / len(train_loader):.4f}")


[Epoch 1] Loss: 0.4933
[Epoch 2] Loss: 0.0844
[Epoch 3] Loss: 0.0575
[Epoch 4] Loss: 0.0457
[Epoch 5] Loss: 0.0356


In [ ]:
# 이제 학습이 끝난 모델의 성능을 평가하는 단계.

correct = 0 #맞힌 샘플 개수
total = 0 # 전체 샘플 개수

with torch.no_grad(): # 가중치 업데이트를 막아놓음.
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = net(images) # 정확히는 N개의 이미지, 10개의 차원이다. 
        _, predicted = torch.max(outputs, 1) # 결과중에 각 이미지마다 가장 큰 값을 고른다.
        total += labels.size(0)
        correct += (predicted == labels).sum().item() # 가장 큰 값이 실제 정답과 맞는지 비교한다.

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 98.77%
